# Algorithm2-人为指定若干关键点

In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 读取两张图片
img1 = cv2.imread('images/image1.jpg')
img2 = cv2.imread('images/image2.jpg')

# 将图片转换为灰度图
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# 手动指定图像1中的关键点坐标 (x, y)
manual_points = [(453, 181), (1257, 121), (1137,1669),(529,1597)]  # 示例坐标

# 将手动指定的点转换为KeyPoint对象
kp1 = [cv2.KeyPoint(x=float(x), y=float(y),size=1) for (x, y) in manual_points]

# 初始化ORB检测器
orb = cv2.ORB_create()
# 计算手动关键点的描述符
kp1, des1 = orb.compute(gray1, kp1)

# 在图像2中检测所有关键点和描述符
sift = cv2.SIFT_create()
kp2, des2 = sift.detectAndCompute(gray2, None)

# 使用FLANN匹配器进行匹配
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# 使用Lowe's ratio test来筛选匹配点
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# 提取匹配点坐标
points1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
points2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

# 计算单应性矩阵
H, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 5.0)
matchesMask = mask.ravel().tolist()

# 使用单应性矩阵对第一张图片进行透视变换
# height, width, channels = image2.shape
# im1_reg = cv2.warpPerspective(image1, H, (width, height))

# 计算关键点位置的变化
point_changes = []
for i, match in enumerate(good_matches):
    if matchesMask[i]:
        pt1 = np.array(kp1[match.queryIdx].pt)
        pt2 = np.array(kp2[match.trainIdx].pt)
        point_changes.append((pt1, pt2, np.linalg.norm(pt1 - pt2)))

# 定义绘制参数
keypoint_size = 10
line_thickness = 5

# 在图片上绘制关键点和匹配线
image1_with_kp = cv2.drawKeypoints(image1, kp1, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
image2_with_kp = cv2.drawKeypoints(image2, kp2, None, color=(0, 255, 0), flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# 绘制匹配线条
for i, match in enumerate(good_matches):
    if matchesMask[i]:
        pt1 = tuple(np.round(kp1[match.queryIdx].pt).astype(int))
        pt2 = tuple(np.round(kp2[match.trainIdx].pt).astype(int) + np.array([image1.shape[1], 0]))
        cv2.circle(image1_with_kp, pt1, keypoint_size, (0, 0, 255), -1)
        cv2.circle(image2_with_kp, pt2 - np.array([image1.shape[1], 0]), keypoint_size, (0, 0, 255), -1)
        cv2.line(image2_with_kp, pt1, pt2 - np.array([image1.shape[1], 0]), (255, 0, 0), line_thickness)

# 合并两张图片用于显示匹配
combined_image = np.hstack((image1_with_kp, image2_with_kp))

# 显示结果
plt.figure(figsize=(20, 10))
plt.title('Keypoints and Matches')
plt.imshow(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))
plt.show()

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\flann\src\miniflann.cpp:522: error: (-215:Assertion failed) query.type() == type && indices.type() == CV_32S && dists.type() == dtype in function 'cv::flann::runKnnSearch_'
